In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import sys

In [3]:
import yaml
import paddle
import paddle.distributed as dist

In [4]:
if '__file__' in globals():
    __dir__ = os.path.dirname(os.path.abspath(__file__))
    sys.path.append(__dir__)
    sys.path.insert(0, os.path.abspath(os.path.join(__dir__, '..')))
else:
    # Handle the case where __file__ is not defined
    pass

In [6]:
from ppocr.data import build_dataloader, set_signal_handlers
from ppocr.modeling.architectures import build_model
from ppocr.losses import build_loss
from ppocr.optimizer import build_optimizer
from ppocr.postprocess import build_post_process
from ppocr.metrics import build_metric
from ppocr.utils.save_load import load_model
from ppocr.utils.utility import set_seed
from ppocr.modeling.architectures import apply_to_static
from ppocr.utils.logging import get_logger
from ppocr.utils.loggers import VDLLogger, WandbLogger, Loggers
from ppocr.utils.utility import print_dict, AverageMeter
import tools.program as program

In [7]:
dist.get_world_size()

1

In [8]:
def test_reader(config, device, logger):
    loader = build_dataloader(config, 'Train', device, logger)
    import time
    starttime = time.time()
    count = 0
    try:
        for data in loader():
            count += 1
            if count % 1 == 0:
                batch_time = time.time() - starttime
                starttime = time.time()
                logger.info("reader: {}, {}, {}".format(
                    count, len(data[0]), batch_time))
    except Exception as e:
        logger.info(e)
    logger.info("finish reader: {}, Success!".format(count))

In [9]:
from argparse import ArgumentParser, RawDescriptionHelpFormatter

In [10]:
def load_config(file_path):
    _, ext = os.path.splitext(file_path)
    assert ext in ['.yml', '.yaml'], "only support yaml files for now"
    config = yaml.load(open(file_path, 'rb'), Loader=yaml.Loader)
    return config

In [11]:
class ArgsParser(ArgumentParser):
    def __init__(self):
        super(ArgsParser, self).__init__(
            formatter_class=RawDescriptionHelpFormatter)
        self.add_argument("-c", "--config", help="configuration file to use")
        self.add_argument(
            "-o", "--opt", nargs='+', help="set configuration options")
        self.add_argument(
            '-p',
            '--profiler_options',
            type=str,
            default=None,
            help='The option of profiler, which should be in format ' \
                 '\"key1=value1;key2=value2;key3=value3\".'
        )

    def parse_args(self, default_config_path, argv=None):
        if 'ipykernel_launcher' in sys.argv[0]:
            args, _ = super(ArgsParser, self).parse_known_args(argv)
            args.config = default_config_path
            args.opt = self._parse_opt(args.opt)
            return args
        
        args = super(ArgsParser, self).parse_args(argv)
        assert args.config is not None, "Please specify --config=configure_file_path."
        args.opt = self._parse_opt(args.opt)
        return args

    def _parse_opt(self, opts):
        config = {}
        if not opts:
            return config
        for s in opts:
            s = s.strip()
            k, v = s.split('=')
            config[k] = yaml.load(v, Loader=yaml.Loader)
        return config


In [12]:
def merge_config(config, opts):
    """
    Merge config into global config.
    Args:
        config (dict): Config to be merged.
    Returns: global config
    """
    for key, value in opts.items():
        if "." not in key:
            if isinstance(value, dict) and key in config:
                config[key].update(value)
            else:
                config[key] = value
        else:
            sub_keys = key.split('.')
            assert (
                sub_keys[0] in config
            ), "the sub_keys can only be one of global_config: {}, but get: " \
               "{}, please check your running command".format(
                config.keys(), sub_keys[0])
            cur = config[sub_keys[0]]
            for idx, sub_key in enumerate(sub_keys[1:]):
                if idx == len(sub_keys) - 2:
                    cur[sub_key] = value
                else:
                    cur = cur[sub_key]
    return config


In [13]:
def check_device(use_gpu, use_xpu=False, use_npu=False, use_mlu=False):
    """
    Log error and exit when set use_gpu=true in paddlepaddle
    cpu version.
    """
    err = "Config {} cannot be set as true while your paddle " \
          "is not compiled with {} ! \nPlease try: \n" \
          "\t1. Install paddlepaddle to run model on {} \n" \
          "\t2. Set {} as false in config file to run " \
          "model on CPU"

    try:
        if use_gpu and use_xpu:
            print("use_xpu and use_gpu can not both be ture.")
        if use_gpu and not paddle.is_compiled_with_cuda():
            print(err.format("use_gpu", "cuda", "gpu", "use_gpu"))
            sys.exit(1)
        if use_xpu and not paddle.device.is_compiled_with_xpu():
            print(err.format("use_xpu", "xpu", "xpu", "use_xpu"))
            sys.exit(1)
        if use_npu:
            if int(paddle.version.major) != 0 and int(
                    paddle.version.major) <= 2 and int(
                        paddle.version.minor) <= 4:
                if not paddle.device.is_compiled_with_npu():
                    print(err.format("use_npu", "npu", "npu", "use_npu"))
                    sys.exit(1)
            # is_compiled_with_npu() has been updated after paddle-2.4
            else:
                if not paddle.device.is_compiled_with_custom_device("npu"):
                    print(err.format("use_npu", "npu", "npu", "use_npu"))
                    sys.exit(1)
        if use_mlu and not paddle.device.is_compiled_with_mlu():
            print(err.format("use_mlu", "mlu", "mlu", "use_mlu"))
            sys.exit(1)
    except Exception as e:
        pass


In [20]:
def preprocess(default_config_path, is_train=False):
    # default_config_path = 'tools/cls_mv3.yml'
    FLAGS = ArgsParser().parse_args(default_config_path)
    profiler_options = FLAGS.profiler_options
    config = load_config(FLAGS.config)
    config = merge_config(config, FLAGS.opt)
    profile_dic = {"profiler_options": FLAGS.profiler_options}
    config = merge_config(config, profile_dic)

    if is_train:
        # save_config
        save_model_dir = config['Global']['save_model_dir']
        os.makedirs(save_model_dir, exist_ok=True)
        with open(os.path.join(save_model_dir, 'config.yml'), 'w') as f:
            yaml.dump(
                dict(config), f, default_flow_style=False, sort_keys=False)
        log_file = '{}/train.log'.format(save_model_dir)
    else:
        log_file = None
    logger = get_logger(log_file=log_file)

    # check if set use_gpu=True in paddlepaddle cpu version
    use_gpu = config['Global'].get('use_gpu', False)
    use_xpu = config['Global'].get('use_xpu', False)
    use_npu = config['Global'].get('use_npu', False)
    use_mlu = config['Global'].get('use_mlu', False)

    alg = config['Architecture']['algorithm']
    assert alg in [
        'EAST', 'DB', 'SAST', 'Rosetta', 'CRNN', 'STARNet', 'RARE', 'SRN',
        'CLS', 'PGNet', 'Distillation', 'NRTR', 'TableAttn', 'SAR', 'PSE',
        'SEED', 'SDMGR', 'LayoutXLM', 'LayoutLM', 'LayoutLMv2', 'PREN', 'FCE',
        'SVTR', 'SVTR_LCNet', 'ViTSTR', 'ABINet', 'DB++', 'TableMaster', 'SPIN',
        'VisionLAN', 'Gestalt', 'SLANet', 'RobustScanner', 'CT', 'RFL', 'DRRG',
        'CAN', 'Telescope', 'SATRN', 'SVTR_HGNet'
    ]

    if use_xpu:
        device = 'xpu:{0}'.format(os.getenv('FLAGS_selected_xpus', 0))
    elif use_npu:
        device = 'npu:{0}'.format(os.getenv('FLAGS_selected_npus', 0))
    elif use_mlu:
        device = 'mlu:{0}'.format(os.getenv('FLAGS_selected_mlus', 0))
    else:
        device = 'gpu:{}'.format(dist.ParallelEnv()
                                 .dev_id) if use_gpu else 'cpu'
    check_device(use_gpu, use_xpu, use_npu, use_mlu)

    device = paddle.set_device(device)

    config['Global']['distributed'] = dist.get_world_size() != 1

    loggers = []

    if 'use_visualdl' in config['Global'] and config['Global']['use_visualdl']:
        save_model_dir = config['Global']['save_model_dir']
        vdl_writer_path = save_model_dir
        log_writer = VDLLogger(vdl_writer_path)
        loggers.append(log_writer)
    if ('use_wandb' in config['Global'] and
            config['Global']['use_wandb']) or 'wandb' in config:
        save_dir = config['Global']['save_model_dir']
        wandb_writer_path = "{}/wandb".format(save_dir)
        if "wandb" in config:
            wandb_params = config['wandb']
        else:
            wandb_params = dict()
        wandb_params.update({'save_dir': save_dir})
        log_writer = WandbLogger(**wandb_params, config=config)
        loggers.append(log_writer)
    else:
        log_writer = None
    print_dict(config, logger)

    if loggers:
        log_writer = Loggers(loggers)
    else:
        log_writer = None

    logger.info('train with paddle {} and device {}'.format(paddle.__version__,
                                                            device))
    return config, device, logger, log_writer


In [21]:
default_config_path = "tools/cls_mv3.yml"
config, device, logger, vdl_writer = preprocess(default_config_path, is_train=True, )

[2023/12/01 09:18:34] ppocr INFO: Architecture : 
[2023/12/01 09:18:34] ppocr INFO:     Backbone : 
[2023/12/01 09:18:34] ppocr INFO:         model_name : small
[2023/12/01 09:18:34] ppocr INFO:         name : MobileNetV3
[2023/12/01 09:18:34] ppocr INFO:         scale : 0.35
[2023/12/01 09:18:34] ppocr INFO:     Head : 
[2023/12/01 09:18:34] ppocr INFO:         class_dim : 2
[2023/12/01 09:18:34] ppocr INFO:         name : ClsHead
[2023/12/01 09:18:34] ppocr INFO:     Neck : None
[2023/12/01 09:18:34] ppocr INFO:     Transform : None
[2023/12/01 09:18:34] ppocr INFO:     algorithm : CLS
[2023/12/01 09:18:34] ppocr INFO:     model_type : cls
[2023/12/01 09:18:34] ppocr INFO: Eval : 
[2023/12/01 09:18:34] ppocr INFO:     dataset : 
[2023/12/01 09:18:34] ppocr INFO:         data_dir : ./train_data/cls
[2023/12/01 09:18:34] ppocr INFO:         label_file_list : ['./train_data/cls/test.txt']
[2023/12/01 09:18:34] ppocr INFO:         name : SimpleDataSet
[2023/12/01 09:18:34] ppocr INFO:   

[2023/12/01 09:18:34] ppocr INFO:     infer_img : doc/imgs_words_en/word_10.png
[2023/12/01 09:18:34] ppocr INFO:     label_list : ['0', '180']
[2023/12/01 09:18:34] ppocr INFO:     log_smooth_window : 20
[2023/12/01 09:18:34] ppocr INFO:     pretrained_model : None
[2023/12/01 09:18:34] ppocr INFO:     print_batch_step : 10
[2023/12/01 09:18:34] ppocr INFO:     save_epoch_step : 3
[2023/12/01 09:18:34] ppocr INFO:     save_inference_dir : None
[2023/12/01 09:18:34] ppocr INFO:     save_model_dir : ./output/cls/mv3/
[2023/12/01 09:18:34] ppocr INFO:     use_gpu : True
[2023/12/01 09:18:34] ppocr INFO:     use_visualdl : False
[2023/12/01 09:18:34] ppocr INFO: Loss : 
[2023/12/01 09:18:34] ppocr INFO:     name : ClsLoss
[2023/12/01 09:18:34] ppocr INFO: Metric : 
[2023/12/01 09:18:34] ppocr INFO:     main_indicator : acc
[2023/12/01 09:18:34] ppocr INFO:     name : ClsMetric
[2023/12/01 09:18:34] ppocr INFO: Optimizer : 
[2023/12/01 09:18:34] ppocr INFO:     beta1 : 0.9
[2023/12/01 09:1

In [26]:
seed = config['Global']['seed'] if 'seed' in config['Global'] else 1024
set_seed(seed)

In [27]:
test_reader(config, device, logger)

[2023/12/01 09:21:19] ppocr INFO: Initialize indexs of datasets:['./train_data/cls/train.txt']


FileNotFoundError: [Errno 2] No such file or directory: './train_data/cls/train.txt'